# Auto_Mobile_Price_Prediction_ML_Model

## Imamdeen
* Unversity of Solent 
* BSc Final year Project 
* M.A.M Imamdeen

## Python Libraries

In [1]:
import pandas as pd
import numpy as np
import  matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pickle

from sklearn.linear_model import LinearRegression
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

## Reading and Displaying the dataset

In [4]:
df = pd.read_csv("Toyota_Vehicle_data.csv")
df

,name,company,year,Price,kms_driven,fuel_type
0,Allion,Toyota,2018,9300000,8000,Petrol
1,Allion,Toyota,2018,9890000,9875,Petrol
2,Allion,Toyota,2018,9850000,10907,Petrol
3,Allion,Toyota,2008,4885000,13000,Petrol
4,Allion,Toyota,2017,8000000,22300,Petrol
...,...,...,...,...,...,...
1494,Vitz,Toyota,2008,4550000,114000,Petrol
1495,Hyundai,Hyundai Accent 2001,2001,1875000,75000,Petrol
1496,Maruti,Maruti Suzuki Omni,2011,1350000,135582,Petrol
1497,Datsun,Datsun Go Plus,2017,2750000,90000,Petrol


## Pre-Processing and Finding out Anomalies

In [5]:
df.shape

(1499, 6)

In [6]:
df["name"].nunique()

39

# Anomalies found in "name" column:
* Names are pretty inconsistent.
* Names have company names attached to it.
* We can consider first 2-3 words from the names as final name, in-order to make the categorical.

In [7]:
df["company"].unique()

array(['Toyota', 'Hyundai Accent 2001', 'Maruti Suzuki Omni',
       'Datsun Go Plus', 'Maruti Suzuki Wagon'], dtype=object)

In [8]:
df["company"].nunique()

5

# Anomalies found in "company" column:
* Some "company" names are incorrect.


In [9]:
df["year"].unique()

array([2018, 2008, 2017, 2016, 2014, 2013, 2011, 2010, 2009, 2007, 2012,
       2006, 2004, 2003, 2002, 2001, 2005, 2015, 2019, 2000, 2020],
      dtype=int64)

In [10]:
 df["year"].nunique()

21

# Anomalies found in "year" column:
* Many garbage/non-year values are present.
* We have convert the dtype of the "year" column from object to int64.

In [11]:
df["Price"].unique()

array([ 9300000,  9890000,  9850000,  4885000,  8000000,  7675000,
        7900000,  7000000, 11000000, 13000000, 14500000, 11500000,
        9700000, 11250000,  8250000,  8100000,  8800000,  7950000,
        9500000,  7350000,  7100000,  7300000,  7290000,  7250000,
        8400000,  7500000,  9100000,  6100000,  8600000,  7650000,
        7750000,  6950000,  5575000,  5275000,  7225000,  5150000,
        5850000,  4900000,  5900000,  8225000,  6400000,  7800000,
        6200000,  9000000,  4500000,  6500000,  6300000,  5800000,
        5500000,  6000000,  5200000,  5350000,  5100000,  8200000,
        6900000,  7390000,  9200000,  6350000,  7600000,  6990000,
        6750000,  8575000,  6600000,  8500000,  5650000,  8950000,
        5600000,  4975000,  7475000,  6195000,  4875000,  6250000,
        5950000,  7200000,  7490000,  5125000,  5700000,  4700000,
        4950000, 11450000,  8300000,  8550000,  5975000,  6775000,
        5750000,  8450000,  6975000,  5775000,  1050000,  8900

In [12]:
df["Price"].nunique()

159

In [13]:
count = 0
for each_price in df["Price"]:
    if each_price == "Ask For Price":
        count += 1
print("No. of records in df with Price as \"Ask For Price\":", count)

No. of records in df with Price as "Ask For Price": 0


## Anomalies found in "Price" column:
- We have to look after the _"Ask for Price"_ type values in the column.
- We have convert the dtype of the "Price" column from _object_ to _int64_ by removing the commas in-between them.

In [14]:
df["kms_driven"].unique()

array([  8000,   9875,  10907,  13000,  22300,  26000,  27000,  31000,
        34000,  41000,  47000,  49000,  54000,  56000,  60000,  71000,
        71500,  75000,  76501,  85000,  86000,  87000,  89000,  90000,
        90080,  92000,  92700,  93000,  96000,  97000, 100000, 102000,
       104000, 105000, 106000, 107000, 109000, 110000, 112000, 112500,
       113000, 115000, 116000, 117000, 118000, 120000, 121000, 124000,
       125000, 126000, 128000, 130000, 133000, 135000, 137000, 138000,
        95000,  99000, 102500, 107960, 122600, 127000, 132000, 143000,
       123000, 124500, 140000, 147000, 148000, 150000, 157000, 159000,
       160000, 174000, 183000, 190000, 200000, 230000,   5800,  15000,
        22000,  28000,  30000,  30400,  32000,  34300,  36000,  38000,
        38750,  39500,  40000,  40500,  42035,  43000,  44000,  45000,
        46000,  47100,  47500,  48000,  53000,  53300,  55000,  55227,
        55400,  57000,  58000,  60100,  62000,  62500,  63000,  63421,
      

In [15]:
df["kms_driven"].nunique()

485

In [16]:
count = 0

for each_kms in df["kms_driven"]:
    if each_kms == "Petrol":
        count += 1
        
print("No. of records in df with kms_driven as \"nan\":", df["kms_driven"].isnull().sum())
print("No. of records in df with kms_driven as \"Petrol\":", count)

No. of records in df with kms_driven as "nan": 0
No. of records in df with kms_driven as "Petrol": 0


# Anomalies found in "kms_driven" column:
* We have to look after the "Petrol" type values in the column.
* We have to convert the dtype of the "kms_driven" column from object with "kms" in the end to int64 by removing the them.

In [17]:
df["fuel_type"].unique()

array(['Petrol', 'Hybrid', 'Electric', 'Diesel'], dtype=object)

In [18]:
df["fuel_type"].nunique()

4

In [19]:
print("\"fuel_type\" column has", df["fuel_type"].isnull().sum(), "null values.")

"fuel_type" column has 0 null values.


## Data Sanity (Clearing) & Filtering

### Converting the "year" column to _int64_ type

In [21]:
df["year"] = pd.to_numeric(df["year"])


### Removing _null_ records from "fuel_type" column

In [27]:
df = df[~df["fuel_type"].isnull()]

### Considering only the first _3 words_ from the "name" column

In [30]:
df["name"] = df["name"].str.split().str.slice(0, 3).str.join(" ")

### Resetting the indexes of the dataframe

In [31]:
df = df.reset_index(drop=True)

In [32]:
df

,name,company,year,Price,kms_driven,fuel_type
0,Allion,Toyota,2018,9300000,8000,Petrol
1,Allion,Toyota,2018,9890000,9875,Petrol
2,Allion,Toyota,2018,9850000,10907,Petrol
3,Allion,Toyota,2008,4885000,13000,Petrol
4,Allion,Toyota,2017,8000000,22300,Petrol
...,...,...,...,...,...,...
1494,Vitz,Toyota,2008,4550000,114000,Petrol
1495,Hyundai,Hyundai Accent 2001,2001,1875000,75000,Petrol
1496,Maruti,Maruti Suzuki Omni,2011,1350000,135582,Petrol
1497,Datsun,Datsun Go Plus,2017,2750000,90000,Petrol


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        1499 non-null   object
 1   company     1499 non-null   object
 2   year        1499 non-null   int64 
 3   Price       1499 non-null   int64 
 4   kms_driven  1499 non-null   int64 
 5   fuel_type   1499 non-null   object
dtypes: int64(3), object(3)
memory usage: 70.4+ KB


In [34]:
df.describe()

,year,Price,kms_driven
count,1499.000000,1.499000e+03,1499.000000
mean,2011.003336,6.390247e+06,94316.575717
std,4.830849,2.091601e+06,49215.086233
min,2000.000000,1.050000e+06,500.000000
25%,2008.000000,5.200000e+06,59583.000000
50%,2012.000000,6.100000e+06,90000.000000
75%,2015.000000,7.500000e+06,123000.000000
max,2020.000000,2.210000e+07,386400.000000


In [35]:
df["Price"].min()

1050000

In [36]:
df["Price"].quantile(0.25)

5200000.0

In [37]:
df["Price"].median()

6100000.0

In [38]:
df["Price"].quantile(0.75)

7500000.0

In [39]:
df["Price"].max()

22100000

In [40]:
find_outlier = df[df["Price"].map(str).apply(len) > 6]

In [41]:
find_outlier["Price"].min()

1050000

In [42]:
find_outlier["Price"].quantile(0.25)

5200000.0

In [43]:
find_outlier["Price"].median()

6100000.0

In [44]:
find_outlier["Price"].quantile(0.75)

7500000.0

In [45]:
find_outlier["Price"].max()

22100000

In [46]:
find_outlier[find_outlier["Price"] > find_outlier["Price"].quantile(0.75)]

,name,company,year,Price,kms_driven,fuel_type
0,Allion,Toyota,2018,9300000,8000,Petrol
1,Allion,Toyota,2018,9890000,9875,Petrol
2,Allion,Toyota,2018,9850000,10907,Petrol
4,Allion,Toyota,2017,8000000,22300,Petrol
5,Allion,Toyota,2016,7675000,26000,Petrol
...,...,...,...,...,...,...
1255,Tank,Toyota,2018,8790000,36500,Petrol
1256,Tank,Toyota,2017,7800000,50000,Petrol
1318,Vitz,Toyota,2019,8790000,1000,Petrol
1319,Vitz,Toyota,2020,8900000,1300,Petrol


In [49]:
outlier = find_outlier[find_outlier["Price"] == 11000000]
outlier

,name,company,year,Price,kms_driven,fuel_type
8,Allion,Toyota,2016,11000000,34000,Petrol


In [50]:
df = df[df["Price"] != 11000000]

In [51]:
df

,name,company,year,Price,kms_driven,fuel_type
0,Allion,Toyota,2018,9300000,8000,Petrol
1,Allion,Toyota,2018,9890000,9875,Petrol
2,Allion,Toyota,2018,9850000,10907,Petrol
3,Allion,Toyota,2008,4885000,13000,Petrol
4,Allion,Toyota,2017,8000000,22300,Petrol
...,...,...,...,...,...,...
1494,Vitz,Toyota,2008,4550000,114000,Petrol
1495,Hyundai,Hyundai Accent 2001,2001,1875000,75000,Petrol
1496,Maruti,Maruti Suzuki Omni,2011,1350000,135582,Petrol
1497,Datsun,Datsun Go Plus,2017,2750000,90000,Petrol


In [52]:
df.to_csv("Cleaned.csv", index=False)

In [53]:
X = df.drop(columns = "Price")
y = df["Price"]

In [54]:
X

,name,company,year,kms_driven,fuel_type
0,Allion,Toyota,2018,8000,Petrol
1,Allion,Toyota,2018,9875,Petrol
2,Allion,Toyota,2018,10907,Petrol
3,Allion,Toyota,2008,13000,Petrol
4,Allion,Toyota,2017,22300,Petrol
...,...,...,...,...,...
1494,Vitz,Toyota,2008,114000,Petrol
1495,Hyundai,Hyundai Accent 2001,2001,75000,Petrol
1496,Maruti,Maruti Suzuki Omni,2011,135582,Petrol
1497,Datsun,Datsun Go Plus,2017,90000,Petrol


In [55]:
y

0       9300000
1       9890000
2       9850000
3       4885000
4       8000000
         ...   
1494    4550000
1495    1875000
1496    1350000
1497    2750000
1498    4425000
Name: Price, Length: 1498, dtype: int64

In [56]:
X_train, X_test,y_train, y_test = train_test_split(X, y, test_size = 0.1)

In [57]:
ohenc = OneHotEncoder()
ohenc.fit(X[['name','company','fuel_type']])

OneHotEncoder()

In [58]:
column_trans = make_column_transformer((OneHotEncoder(categories = ohenc.categories_), ['name','company','fuel_type']), remainder='passthrough')

In [59]:
lnregr = LinearRegression()

In [60]:
final = make_pipeline(column_trans, lnregr)

In [61]:
final.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Allion', 'Allion 240', 'Allion 260', 'Aqua', 'Aqua G Grade',
       'Aqua G Sport', 'Aqua G grade', 'Aqua S Grade', 'Aqua X Urban',
       'Avanza', 'Axio', 'Axio Fielder', 'Axio G Grade', 'Axio WXB',
       'Axio X Grade', 'Belta', 'Carina', 'Corolla 110', 'Corol...
       'Prius 3rd Generation', 'Prius S Grade', 'Prius S Touring',
       'Roomy', 'Soluna', 'Tank', 'Vios', 'Vitz', 'soluna'], dtype=object),
                                                                            array(['Datsun Go Plus', 'Hyundai Accent 2001', 'Maruti Suzuki Omni',
       'Maruti Suzuki Wagon', 'Toyota'], dtype=object),
                                                                            array(['Diesel', 'Electric', 'Hybrid', 'Petrol'], dtype=object

In [62]:
final.score(X_test, y_test)

0.847695981462808

In [63]:
y_pred = final.predict(X_test)

In [64]:
y_pred

array([ 7620285.71108878,  7924538.00673687,  7785461.39964223,
        6002497.8868705 ,  6797836.39028859,  5273316.15140271,
        5384930.81851518,  5679587.24032712,  4584652.21648419,
        7537538.46887767,  3369749.24162161,  7392591.26514578,
        3529453.1419034 ,  7832795.30188668,  6192315.68563688,
        5501731.39848232, 11617234.62690794,  3557671.38559544,
        3606605.6206249 ,  7219085.52870214,  5554804.94100213,
        5718693.4554441 ,  5202194.53438401,  5105403.01582742,
        6424331.51629233,  3843837.76442194,  3957631.48640394,
        9151757.32177901,  5831009.21277082,  8991409.22500455,
        6759236.76760924,  4572418.65772676,  8036677.42112458,
        6886639.63547635,  4079967.07397759,  7010077.00272   ,
        5360132.18555343,  7854806.72181988, 11512026.02159464,
        7282945.89075148,  8102756.085127  ,  4877759.97388816,
        6420661.44866514,  7893963.09567344,  6613587.30746865,
        6778091.86898506,  7148353.45316

In [65]:
r2_score(y_test, y_pred)

0.847695981462808

In [66]:
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
    
    lnregr = LinearRegression()
    
    final = make_pipeline(column_trans, lnregr)
    
    final.fit(X_train, y_train)
    
    y_pred = final.predict(X_test)
    
    print(r2_score(y_test, y_pred))

0.778498136904836
0.8600411341190513
0.8362055874839817
0.8209530865778298
0.8491852851753623
0.8598937459410623
0.887863974969675
0.8516680832694452
0.86064759816181
0.8388100298367549


In [67]:
scores = []
for i in range(1000):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = i)
    
    lnregr = LinearRegression()
    
    final = make_pipeline(column_trans, lnregr)
    
    final.fit(X_train, y_train)
    
    y_pred = final.predict(X_test)
    
    scores.append(r2_score(y_test, y_pred))

In [68]:
np.argmax(scores)

157

In [71]:
scores[np.argmax(scores)]

0.908471241197648

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = np.argmax(scores))
    
lnregr = LinearRegression()
    
final = make_pipeline(column_trans, lnregr)
    
final.fit(X_train, y_train)
    
y_pred = final.predict(X_test)
    
r2_score(y_test, y_pred)

0.908471241197648

In [73]:
np.sqrt(MSE(y_test, y_pred))

676640.04010154

In [74]:
y_pred

array([ 2808657.15682721,  8203738.98336983,  4842448.92357898,
        6962942.76279044,  5359160.68154919,  5157972.44629633,
        3336072.57063508,  4227813.79839969,  3948322.42825449,
        8964095.31892109, 16048031.15967214,  5018072.48422158,
        5692270.17811835,  3624119.47968507,  2814937.86177433,
        7042491.1732161 ,  5516470.76173282,  5855055.42842507,
        5911156.9561938 ,  6215258.06780994,  6340157.98471367,
        7717010.8280617 ,  9160876.89662957,  8340800.41960204,
        7432318.1823678 ,  4058234.76482844,  6744744.7924093 ,
        7404055.01010585,  6001512.99588871,  6039197.22557116,
        5238768.18153417,  3848773.06832361,  6050188.45922852,
        7267144.31133962,  6868287.58055949,  5392448.41776872,
        6890162.32189417,  7288926.94151556,  5543163.7577579 ,
        7718445.11710417,  5705738.00823236,  6234100.18265116,
        5183675.30041087,  5244262.09098363,  7942872.59298253,
        7773125.97088933,  6465806.37600

In [75]:
final

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Allion', 'Allion 240', 'Allion 260', 'Aqua', 'Aqua G Grade',
       'Aqua G Sport', 'Aqua G grade', 'Aqua S Grade', 'Aqua X Urban',
       'Avanza', 'Axio', 'Axio Fielder', 'Axio G Grade', 'Axio WXB',
       'Axio X Grade', 'Belta', 'Carina', 'Corolla 110', 'Corol...
       'Prius 3rd Generation', 'Prius S Grade', 'Prius S Touring',
       'Roomy', 'Soluna', 'Tank', 'Vios', 'Vitz', 'soluna'], dtype=object),
                                                                            array(['Datsun Go Plus', 'Hyundai Accent 2001', 'Maruti Suzuki Omni',
       'Maruti Suzuki Wagon', 'Toyota'], dtype=object),
                                                                            array(['Diesel', 'Electric', 'Hybrid', 'Petrol'], dtype=object

In [76]:
pickle.dump(final, open("Price_Prediction.pkl", "wb"))